In [1]:
import json
import logging
import os
import sys

import pandas as pd

from data_reader import DataReader
from finder import find_layers
from writexl import write_layers

pd.set_option('display.max_rows', 10000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [2]:
# конвертация путей файлов в зависимости от системы
def replace_slash(file_path):
    platform = sys.platform
    slash_map = {'win32': '\\',
                 'cygwin': '\\',
                 'darwin': '/',
                 'linux2': '/'}
    if platform not in slash_map.keys(): platform = 'linux2'
    return file_path.replace('\\', slash_map[platform])


# очистка папки output_folder
def clear_out_folder(output_folder):
    files = os.listdir(output_folder)
    for f in files:
        path_dir = replace_slash(output_folder + "\\" + f)
        os.remove(path_dir)


In [3]:
input_folder = "input_data"
out_folder = "output_data"
output_path_l = replace_slash(out_folder + "\\" + "non_perf_layers.xlsx")
output_path_a = replace_slash(out_folder + "\\" + "act_perf.xlsx")

if not os.path.exists(out_folder):
    os.makedirs(out_folder)
else:
    clear_out_folder(out_folder)

out_path_log = replace_slash(out_folder + "\\" + "Report.txt")
logging.basicConfig(format=u'%(levelname)-8s : %(message)s', filename=out_path_log, filemode='w')

with open("config.json", 'r') as f:
    try:
        conf = json.load(f)
        SOIL_CUT = float(conf["SOIL_CUT"])
        perf_path = conf["perf_path"]
        fes_path = conf["fes_path"]
    except BaseException as e:
        logging.error("Error loading config file. " + str(e))
        sys.exit()
perf_path = replace_slash(input_folder + '\\' + perf_path)
fes_path = replace_slash(input_folder + '\\' + fes_path)

dr = DataReader()
try:
    perf_ints = dr.perf_reader(perf_path)
except BaseException as e:
    logging.error("Error loading perf file. " + str(e))
    sys.exit()
try:
    fes_dict = dr.fes_reader(fes_path)
except BaseException as e:
    logging.error("Error loading fes file. " + str(e))
    sys.exit()

perf_rig_diff, rig_perf_diff = dr.well_diff()
if len(perf_rig_diff) > 0:
    logging.warning("These wells in perf file are absent in rigis "
                    + str(perf_rig_diff))
if len(rig_perf_diff) > 0:
    logging.warning("These wells in rigis file are absent in perf "
                    + str(rig_perf_diff))

try:
    lost_layers = find_layers(perf_ints, fes_dict, SOIL_CUT)
except BaseException as e:
    logging.error("Error while finding layers " + str(e))
    sys.exit()

started reading perf xl
done reading perf xl and started processing perf data
started reading fes xl


100%|██████████| 1579/1579 [00:00<00:00, 20743.85it/s]

done reading fes xl
done processing data



100%|██████████| 1574/1574 [00:00<00:00, 55810.59it/s]


In [7]:
try:
    write_layers(output_path_l, lost_layers)
except BaseException as e:
    logging.error("Error while writing results " + str(e))
    sys.exit()


100%|██████████| 6010/6010 [00:00<00:00, 6169.05it/s]


In [3]:
dw = pd.read_excel('output_data/wells_diff.xlsx')

In [4]:
dw

,нет в перф,нет в ригис,отсут. н-нас.,Неэксп
0,17802,32653,27061,1М
1,20890/2,32753,15641,1Т
2,1047,15743,1974,25
3,27331,32314,168,31
4,281,20477,59,32
5,20477/1,32333,8165,36
6,8735,32420,2276,38
7,32443/1,32651,46э,46
8,27008/2,32673,27080,47
9,32314/1,32443,16465,53Э


In [11]:
dw.cloumns = [0, 1, 2, 3]

In [26]:
es = []
for e in dw['Неэксп'].unique():
    if e in dw['нет в перф'].unique():
        es.append(e)
    if e in dw['нет в ригис'].unique():
        print(0)
        es.append(e)

In [20]:
dw['Неэксп'].unique()

array(['1м', '1т', '25', ..., '8021д', '11982', '303'], dtype=object)

In [19]:
dw['Неэксп'] = dw['Неэксп'].str.lower()

In [24]:
es

['1м',
 '28',
 '47э',
 '15617',
 '15658',
 '17802',
 '3',
 '7',
 '8',
 '10',
 '16408',
 '17439',
 '17441',
 '17452',
 '17475',
 '17495',
 '21159б',
 '27011',
 '57а',
 '165а',
 '167а',
 '170а',
 '531',
 '609',
 '734',
 '746',
 '757',
 '786',
 '787',
 '791',
 '793',
 '795',
 '800р',
 '801',
 '1001',
 '1024',
 '1030',
 '1047',
 '5913',
 '8105',
 '8735',
 '8753',
 '13447',
 '13571',
 '15117',
 '21534',
 '21585',
 '21628',
 '21776',
 '25494',
 '27327',
 '27331',
 '16п',
 '118а',
 '281',
 '5009',
 '17821',
 '17826',
 '17829',
 '17837',
 '17840',
 '17841',
 '26893а',
 '26989',
 '487',
 '70']

In [25]:
len(es)

66

In [30]:
un = list(set(list(dw['нет в перф'].unique())).difference(es))

In [34]:
lstr = '\n'.join(un[1:])
# for l in un:
#     lstr += l + '\n'

In [35]:
lstr

'32322/2\n20856/1\n32372/1\n21003/2\n32673/2\n20858/2\n21003/1\n32673/1\n29769\n32653/2\n32420/2\n32651/2\n32322/1\n14861/1\n32443/2\n20858/1\n32280/2\n32378/2\n32280/1\n32420/1\n32653/1\n32442/1\n32372/2\n20890/1\n32333/1\n3472д/1\n32333/2\n20836/2\n3472д/2\n32485/2\n20963/2\n32443/1\n26849/2\n25433/2\n32304/1\n20477/2\n32753/1\n25433/1\n20963/1\n20613\n15743/1\n32442/2\n26849/1\n32485/1\n32304/2\n20856/2\n27008/1\n32753/2\n27008/2\n32314/1\n32651/1\n32378/1\n20477/1\n20890/2'

In [36]:
with open('unknown.csv', 'w') as f:
    f.write(lstr)